In [1]:
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

from transformers import (
    DataCollatorWithPadding,
    AutoModel,
    AutoTokenizer
)
from datasets import Dataset

os.chdir("/g/data/jr19/rh2942/text-empathy/")
from evaluation import pearsonr
from utils.utils import plot, get_device, set_all_seeds

In [2]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false' # due to huggingface warning
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
class DataModule():
    def __init__(self, task, checkpoint, batch_size, max_len, feature_to_tokenise):
        super(DataModule, self).__init__()
        self.task = task
        self.checkpoint = checkpoint
        self.batch_size = batch_size
        self.max_len = max_len
        self.tokeniser = AutoTokenizer.from_pretrained(
            self.checkpoint,
            use_fast=True
        )
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokeniser)
        self.feature_to_tokenise = feature_to_tokenise # to tokenise function
    
    def _process_raw(self, path, send_label):
        data = pd.read_csv(path, sep='\t')
    
        if send_label:
            data = data[self.feature_to_tokenise + [self.task]]
        else:
            data = data[self.feature_to_tokenise]
            
        return data

    def _tokeniser_fn(self, sentence):
        if len(self.feature_to_tokenise) == 1: # only one feature
            return self.tokeniser(sentence[self.feature_to_tokenise[0]], truncation=True)
        # otherwise tokenise a pair of sentence
        return self.tokeniser(sentence[self.feature_to_tokenise[0]], sentence[self.feature_to_tokenise[1]], truncation=True)

    def _process_input(self, file, send_label):
        data = self._process_raw(path=file, send_label=send_label)
        data = data.reset_index(drop=True)
        data = Dataset.from_pandas(data, preserve_index=False) # convert to huggingface dataset
        data = data.map(self._tokeniser_fn, batched=True, remove_columns=self.feature_to_tokenise) # tokenise
        data = data.with_format('torch')
        return data

    def dataloader(self, file, send_label, shuffle):
        data = self._process_input(file=file, send_label=send_label)
        return DataLoader(
            data,
            batch_size=self.batch_size,
            shuffle=shuffle,
            collate_fn=self.data_collator,
            num_workers=24
        )

In [4]:
class SingleEncoder(nn.Module):
    def __init__(self, checkpoint):
        super(SingleEncoder, self).__init__()
        self.transformer = AutoModel.from_pretrained(checkpoint)
        self.dropout = nn.Dropout(0.2)
        
        self.pre_classifier = nn.Linear(768, 768)
        
        self.classifier = nn.Linear(768, 512)
        self.pre_final = nn.Linear(512, 256)
        self.final = nn.Linear(256, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        # token_type_ids=None,
    ):

        output = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # token_type_ids=token_type_ids
        )
        
        output = output[0]
        output = output[:, 0]

        output = self.pre_classifier(output)
        
        output = nn.ReLU()(output)
        output = self.dropout(output)
        output = self.classifier(output)
        output = nn.ReLU()(output)
        output = self.pre_final(output)
        output = nn.ReLU()(output)
        output = self.final(output)
        return output

In [5]:
class Trainer:
    def __init__(self, task, model, lr):
        self.device = get_device(0)
        
        self.task = task
        self.model = model.to(self.device)
        
        self.lr = lr
        self.loss_fn = nn.MSELoss()
        
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr = self.lr)
        
        self.best_pearson_r = -1.0 # initiliasation

    def _training_step(self, epoch, train_loader):
        tr_loss = 0.0
        
        self.model.train()
    
        for i, data in enumerate(train_loader, 0):
            input_ids = data['input_ids'].to(self.device, dtype=torch.long)
            attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
            # token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
            targets = data[self.task].to(self.device, dtype=torch.float).view(-1, 1)
            
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                # token_type_ids=token_type_ids,
            )          
            loss = self.loss_fn(outputs, targets)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            tr_loss += loss.item()
    
            if i % 50 == 0:
                loss_step = tr_loss / (i+1)
                print(f'Training loss per 50 steps: {loss_step}')
    
        epoch_loss = tr_loss / (i+1)
        print(f'Epoch: {epoch+1}')
        print(f'The total loss: {epoch_loss}')

    def fit(self, n_epochs, train_loader, dev_loader, output_label, save_model=False):
        set_all_seeds(0)
        for epoch in range(n_epochs):
            self._training_step(epoch, train_loader)

            preds = self.evaluate(dataloader=dev_loader, load_model=False)
            if self.task == 'empathy':
                true = output_label.iloc[:, 0].tolist()
            if self.task == 'distress':
                true = output_label.iloc[:, 1].tolist()
            pearson_r = pearsonr(true, preds)
            print(f'Pearson r: {pearson_r}')
            
            if save_model and (pearson_r > self.best_pearson_r):
                self.best_pearson_r = pearson_r   
                torch.save(self.model.state_dict(), 'roberta-empathy.pth')
                print("Saved the model in epoch " + str(epoch+1))

        torch.cuda.empty_cache()

    def evaluate(self, dataloader, load_model=False):
        if load_model:
            self.model.load_state_dict(torch.load('roberta-empathy.pth'))
    
        pred = torch.empty((len(dataloader.dataset), 1), device=self.device) # len(self.dev_loader.dataset) --> # of samples
        
        self.model.eval()
    
        with torch.no_grad():
            idx = 0
            for data in dataloader:
                input_ids = data['input_ids'].to(self.device, dtype=torch.long)
                attention_mask = data['attention_mask'].to(self.device, dtype=torch.long)
                # token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
                
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    # token_type_ids=token_type_ids,
                )

                batch_size = outputs.shape[0]
                pred[idx:idx+batch_size, :] = outputs
                idx += batch_size
            
        return [float(k) for k in pred]

In [6]:
MAX_LEN = 256
BATCH_SIZE = 8
lr = 1e-05
checkpoint = 'roberta-base'
task_list = ['empathy', 'distress']

train_file = './data/PREPROCESSED-WS22-WS23-train.tsv'

# WASSA 2022
# dev_file = './data/PREPROCESSED-WS22-dev.tsv'
# dev_label_file = './data/WASSA22/goldstandard_dev_2022.tsv'
# test_file = './data/PREPROCESSED-WS22-test.tsv'

# WASSA 2023
dev_file = './data/PREPROCESSED-WS23-dev.tsv'
dev_label_file = './data/WASSA23/goldstandard_dev.tsv'
test_file = './data/PREPROCESSED-WS23-test.tsv'

In [7]:
data_module = DataModule(
    task=task_list[0],
    checkpoint=checkpoint,
    batch_size=BATCH_SIZE,
    max_len=MAX_LEN,
    feature_to_tokenise=['demographic_essay', 'article']
)

In [8]:
train_loader = data_module.dataloader(file=train_file, send_label=True, shuffle=True)
dev_loader = data_module.dataloader(file=dev_file, send_label=False, shuffle=False)

Map:   0%|          | 0/2636 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [9]:
dev_label = pd.read_csv(dev_label_file, sep='\t', header=None)

In [10]:
model = SingleEncoder(checkpoint=checkpoint)

trainer = Trainer(
    task=task_list[0],
    model=model,
    lr=lr
)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
trainer.fit(n_epochs=8, train_loader=train_loader, dev_loader=dev_loader,
            output_label=dev_label, save_model=False)

Training loss per 50 steps: 10.632017135620117
Training loss per 50 steps: 14.69898635265874
Training loss per 50 steps: 11.864423293878536
Training loss per 50 steps: 9.437687400161035
Training loss per 50 steps: 8.015810783822738
Training loss per 50 steps: 7.121970001919811
Training loss per 50 steps: 6.587764896823718
Epoch: 1
The total loss: 6.328615145972281
Pearson r: 0.197
Training loss per 50 steps: 4.39067268371582
Training loss per 50 steps: 3.508858456331141
Training loss per 50 steps: 3.50756897696174
Training loss per 50 steps: 3.581367394189961
Training loss per 50 steps: 3.5898176075807258
Training loss per 50 steps: 3.567614012030491
Training loss per 50 steps: 3.5955927181877567
Epoch: 2
The total loss: 3.6091153928727815
Pearson r: 0.225
Training loss per 50 steps: 4.518770694732666
Training loss per 50 steps: 3.5791295813579187
Training loss per 50 steps: 3.3385148768377775
Training loss per 50 steps: 3.3210338314637444
Training loss per 50 steps: 3.2169814006013064

In [42]:
test_loader = data_module.dataloader(file=test_file, send_label=False, shuffle=False)
pred = trainer.evaluate(dataloader=test_loader, load_model=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [43]:
pred_df = pd.DataFrame({'emp': pred, 'dis': pred})

In [44]:
pred_df

,emp,dis
0,4.743536,4.743536
1,5.375017,5.375017
2,5.558261,5.558261
3,4.441684,4.441684
4,4.513950,4.513950
...,...,...
95,5.794048,5.794048
96,5.106700,5.106700
97,3.298420,3.298420
98,4.998303,4.998303


In [45]:
pred_df.to_csv('./tmp/predictions_EMP.tsv', sep='\t', index=None, header=None)

# Extra

## Combined guide and final

In [28]:
class GuidedRegression(nn.Module):
    def __init__(self, checkpoint):
        super(GuidedRegression, self).__init__()
        self.transformer = AutoModel.from_pretrained(checkpoint)
        self.dropout = nn.Dropout(0.2)
        
        self.pre_classifier = nn.Linear(768, 768)
        self.pre_classifier_guide = nn.Linear(769, 768)
        
        self.classifier = nn.Linear(768, 512)
        self.pre_final = nn.Linear(512, 256)
        self.final = nn.Linear(256, 1)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        # token_type_ids=None,
        guide=None
    ):

        output = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # token_type_ids=token_type_ids
        )
        
        output = output[0]
        output = output[:, 0]

        if guide is None:
            output = self.pre_classifier(output)
        else:
            output = torch.cat([output, guide], 1)
            output = self.pre_classifier_guide(output)
        
        output = nn.ReLU()(output)
        output = self.dropout(output)
        output = self.classifier(output)
        output = nn.ReLU()(output)
        output = self.pre_final(output)
        output = nn.ReLU()(output)
        output = self.final(output)
        return output